# Part I. Ingesting data.

Create a dictionary with movies for each genre.

In [1]:
import numpy as np
import math
import time
import matplotlib.pyplot as plt
import random

from scipy import stats

### Read in the ratings. Ratings are in the format - userId,movieId,rating

In [2]:
ratingsFile = open("ratingsSummary.csv", "r")

userRatings = dict()
relevantMovies = dict()

for line in ratingsFile:
    data = map(float,line.split(','))
    
    usr = int(data[0])
    mov = int(data[1])
    r = int(data[2])
    
    if usr in userRatings:
        userRatings[usr][mov] = r
    else:
        userRatings[usr] = dict()
        userRatings[usr][mov] = r
        
    relevantMovies[mov] = True
    
print len(userRatings.keys()), len(relevantMovies.keys())

200 2493


### Now, read in the movies.

In [3]:
movies = dict() # a category -> list_of_movies dict stored as integers
movieCats = dict() # movieID -> categories movie is part

# catID returns the index of a category of type string. catName returns the name of the category given its ID.
catID = dict()
catName = dict()

# same as with cat
movieID = dict()
movieName = dict()

In [4]:
allData = open("movies.csv", "r")

# this first line contains header info
allData.readline()

numMovies = 0
numGenres = 0

while True:
    line = allData.readline()
    
    if line == '':
        break
    
    curMovieID = int(line.split(",", 1)[0])
    if curMovieID not in relevantMovies.keys():
        continue
    
    curMovieName = (line.split(",", 1)[1]).rsplit(",", 1)[0]
    curCategories = line.rsplit(",", 1)[1].rsplit("\r")[0].split("|")
    
    
    # update catID, catName, movieID, movieName
    movieID[curMovieName] = curMovieID
    movieName[curMovieID] = curMovieName
    
    for cat in curCategories:
        if not (cat in catID):
            catID[cat] = numGenres
            catName[numGenres] = cat
            
            numGenres = numGenres + 1
            
    movieCats[curMovieID] = curCategories
            
    for cat in curCategories:
        if catID[cat] in movies:
            movies[catID[cat]].append(movieID[curMovieName])
        else:
            movies[catID[cat]] = [movieID[curMovieName]]
    
    numMovies = numMovies + 1

print "we have", numMovies, "movies"

we have 2493 movies


In [5]:
print movies[catID['Horror']][:10]
print movieName[movies[catID['Horror']][0]]
print movieCats[movies[catID['Horror']][0]]
print catID.keys()

[593, 841, 1076, 1200, 1214, 1215, 1219, 1241, 1258, 1261]
"Silence of the Lambs, The (1991)"
['Crime', 'Horror', 'Thriller']
['Mystery', 'Romance', 'Sci-Fi', 'Musical', 'Film-Noir', 'Crime', 'Drama', 'Fantasy', 'Western', 'Animation', 'War', 'Adventure', 'Horror', 'Action', '(no genres listed)', 'Comedy', 'Documentary', 'Children', 'Thriller', 'IMAX']


In [6]:
print 'We have', len(userRatings.keys()), 'users'
print 'We have', len(movieCats), 'movies'
print 'We have', len(catID.keys()), 'genres'

We have 200 users
We have 2493 movies
We have 20 genres


In [7]:
numMovies = len(movieCats)
numUsers = len(userRatings.keys())

print numMovies, numUsers

2493 200


#### compute the weights for each user

In [8]:
weights = dict()
for user in userRatings.keys():
    weights[user] = dict()
    
    for genre in catID.keys():
        weights[user][genre] = 0
        
    tot = 0
        
    for mov in userRatings[user]:
        genres = movieCats[mov]
        
        for genre in genres:
            weights[user][genre] = weights[user][genre] + 1
            tot = tot + 1
            
    for genre in catID.keys():
        weights[user][genre] = 1.0 * weights[user][genre] / tot

    # verify that they sum to 1
    curSum = 0
    for genre in weights[user]:
        curSum = curSum + weights[user][genre]
    assert 0.9999 <= curSum and curSum <= 1.0001, "weights don't sum up to 1"

# Part II. Fix k = 3. Increase l from 5 to 50

In [9]:
k = 3

In [10]:
from replacementGreedy import replacementGreedy
from greedysum import gsWrapper
from greedymerge import gmWrapper
from localsearch import lsWrapper

In [11]:
def computeCostOuter(userIndex, A):
    tot = 0

    curUser = userRatings.keys()[userIndex]

    # make sure we are only considering movies the current user rated
    ratedA = list(set(A).intersection(userRatings[curUser].keys()))

    catA = dict()
    for cat in catID.keys():
        catA[cat] = []
    for mov in ratedA:
        for cat in movieCats[mov]:
            catA[cat].append(mov)

    for cat in catID.keys():
        # now, find the highest rated movie in each category
        highestRated = 0
        for mov in catA[cat]:
            highestRated = max(highestRated, userRatings[curUser][mov])

        tot = tot + weights[curUser][cat] * highestRated

    return tot

# compute the greedy maximization solution for S for the second stage submodular maximization
def greedyOuter(userIndex, S):
    greedyS = []

    use = [False for s in S]

    for times in range(k):
        # at each step, add the element that gives the greatest marginal gain 

        bestInd = -1
        bestCost = -1

        for ind in range(len(S)):
            if use[ind] == False:
                greedyS.append(S[ind])

                curCost = computeCostOuter(userIndex, greedyS)
                if curCost > bestCost:
                    bestCost = curCost
                    bestInd = ind

                greedyS.pop()

        greedyS.append(S[bestInd])
        use[bestInd] = True

    return computeCostOuter(userIndex, greedyS)

# return the mean, the confidence interval, and the runtime to compute
def computeRatio(S, otherUsers):
    rtGS = time.time()
    groundSetMean = 0
    for other in otherUsers:
        groundSetMean = groundSetMean + greedyOuter(other, movieName.keys())
    groundSetMean = 1.0 * groundSetMean / len(otherUsers)
    rtGS = time.time() - rtGS
    
#     SMean = 0
#     for other in otherUsers:
#         SMean = SMean + greedyOuter(other, S)
#     SMean = 1.0 * SMean / len(otherUsers)
    
#     print SMean / groundSetMean
#     return SMean / groundSetMean

    rtS = time.time()
    Sval = []
    for other in otherUsers:
        curVal = greedyOuter(other, S) / groundSetMean
        Sval.append(curVal)
    rtS = time.time() - rtS
    
    print np.mean(Sval)
    
    return np.mean(Sval), \
        stats.norm.interval(0.95, loc=np.mean(Sval), scale=np.std(Sval)/np.sqrt(len(otherUsers))), \
        rtS / rtGS

In [ ]:
# return solutionSet, solutionValue, 

In [12]:
solution = []
runtime = []
runtimeRatio = []
calls = []
ratios = []

# Lvalues = [3,4]
Lvalues = [3,4,5,7,10,14,17]
for l in range(20,50,10):
    Lvalues.append(l)

# now, pick 20 random users from the remaining
otherUsers = random.sample([x for x in range(numUsers/2, numUsers)], 20)
    
for l in Lvalues:
    rg = replacementGreedy(numMovies, numUsers/2, l, k, userRatings, weights, movieCats, catID.keys())
    gs = gsWrapper(numMovies, numUsers/2, l, k, userRatings, weights, movieCats, catID.keys())
    gm = gmWrapper(numMovies, numUsers/2, l, k, userRatings, weights, movieCats, catID.keys())
    
    curSol = []
    curRt = []
    curRtRatio = []
    curRatio = []
    
    curCalls = []
    
    for alg in [rg, gs, gm]:
        start = time.time()
        
        algS, algCost, algEvals = alg(movieName.keys())
        
        if alg == rg:
            print 'Replacement Greedy gives cost', algCost
        
        curSol.append(algCost)
        curRt.append(time.time() - start)
        curCalls.append(algEvals)
        
        Smean, ci, rtratio = computeRatio(algS, otherUsers)
        
        curRtRatio.append(rtratio)
        curRatio.append((Smean, ci))
        
    solution.append(curSol)
    runtime.append(curRt)
    runtimeRatio.append(curRtRatio)
    ratios.append(curRatio)
    calls.append(curCalls)
    
    print curSol
    print curRt
    print curRtRatio
    print curRatio
    
    print ""
    print "Done for l = ", l
    print "\n\n\n"

Replacement Greedy gives cost 352.480436783
0.786335657055
Greedy Sum gives cost =  352.480436783
0.786335657055
Greedy Merge gives cost =  416.830681304
Size of S is  119
0.984309141592
[352.48043678347165, 352.48043678347165, 416.8306813043843]
[66.90506100654602, 35.590461015701294, 33.581660985946655]
[0.0010294293268442794, 0.0010279103970716635, 0.04939232558166809]
[(0.7863356570548824, (0.73124797092203075, 0.84142334318773404)), (0.7863356570548824, (0.73124797092203075, 0.84142334318773404)), (0.98430914159247307, (0.9487884013561847, 1.0198298818287614))]

Done for l =  3




Replacement Greedy gives cost 363.71278472
0.82819172809
Greedy Sum gives cost =  362.235155977
0.807805323961
Greedy Merge gives cost =  416.830681304
Size of S is  119
0.984309141592
[363.7127847197164, 362.2351559774917, 416.8306813043843]
[118.82006597518921, 48.83973002433777, 33.689528942108154]
[0.001491920829211572, 0.001532465324974765, 0.04890052236348306]
[(0.82819172808998209, (0.78075016191

In [ ]:
# store locally
filename = open('../data/movielens-sublinear.txt', 'w')

print>>filename, 'solution'
for item in solution:
    for (meanVal,sdVal) in item:
        print>>filename,meanVal,sdVal[0],sdVal[1]
    print>>filename,''
    
print>>filename, 'runtime'
for item in runtime:
    print>>filename,item[0],item[1],item[2],item[3]
    
print>>filename, 'runtimeRatio'
for item in runtimeRatio:
    print>>filename,item[0],item[1],item[2],item[3]
    
print>>filename, 'calls'
for item in calls:
    print>>filename,item[0],item[1],item[2],item[3]

In [18]:
plt.clf()

ax = plt.subplot(111)

fs = 17

# http://matplotlib.org/users/text_intro.html
ax.set_xlabel('l', fontsize=fs)
ax.set_ylabel('Objective Value', fontsize=fs)

plt.ylim(350,430)
plt.xlim(min(Lvalues), max(Lvalues))

xticks = [min(Lvalues)]
for l in range(10, max(Lvalues) + 10, 10):
    xticks.append(l)

ax.set_xticks(xticks)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(fs) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(fs) 

colors = ['ro', 'b^', 'gv']
labelNames = ['Replacement Greedy', 'Greedy Sum', 'Greedy Merge']

for ind in range(len(colors)-1, -1,-1):
    vals = []
    for i in range(len(solution)):
        vals.append(solution[i][ind])
        
    plt.plot(Lvalues, vals, c = colors[ind][0], marker = colors[ind][1], linewidth=2, label = labelNames[ind])

# http://matplotlib.org/1.3.0/examples/pylab_examples/legend_demo.html
legend = ax.legend(loc='lower right')

# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize(fs)

# plt.show()
plt.savefig("../../writeup/images/movielens-sublinear-fixed-k")

plt.close()

In [14]:
plt.clf()

ax = plt.subplot(111)

fs = 17

# http://matplotlib.org/users/text_intro.html
ax.set_xlabel('l', fontsize=fs)
ax.set_ylabel('Ratio', fontsize=fs)

plt.ylim(0.75,1.05)
plt.xlim(min(Lvalues), max(Lvalues))

xticks = [min(Lvalues)]
for l in range(10, max(Lvalues) + 10, 10):
    xticks.append(l)

ax.set_xticks(xticks)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(fs) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(fs) 

colors = ['ro', 'b^', 'gv']
labelNames = ['Replacement Greedy', 'Greedy Sum', 'Greedy Merge']

for ind in range(len(colors)-1, -1,-1):
    vals = []
    for i in range(len(ratios)):
        vals.append(ratios[i][ind][0])
        
    errTop = []
    errBot = []
    for i in range(len(solution)):
        errBot.append(vals[i] - ratios[i][ind][1][0])
        errTop.append(ratios[i][ind][1][1] - vals[i])
        
    plt.plot(Lvalues, vals, c = colors[ind][0], marker = colors[ind][1], linewidth=2, label = labelNames[ind])
    plt.errorbar(Lvalues, vals, yerr = [errBot, errTop], fmt = colors[ind])
    
# http://matplotlib.org/1.3.0/examples/pylab_examples/legend_demo.html
legend = ax.legend(loc='lower right')

# Set the fontsize
# for label in legend.get_texts():
#     label.set_fontsize(fs)

# plt.show()
plt.savefig("../../writeup/images/movielens-sublinear-ratio-fixed-k")

plt.close()

In [15]:
runtime

[[66.90506100654602, 35.590461015701294, 33.581660985946655],
 [118.82006597518921, 48.83973002433777, 33.689528942108154],
 [171.38985204696655, 64.97216701507568, 33.63596796989441],
 [279.3140399456024, 92.03704190254211, 32.58375310897827],
 [436.50922107696533, 137.23782801628113, 32.505358934402466],
 [644.1359438896179, 207.25518918037415, 32.33085584640503],
 [790.4963359832764, 260.4526557922363, 31.5134060382843],
 [933.5082900524139, 322.37378120422363, 31.53041911125183],
 [1449.0383110046387, 533.5977628231049, 31.312901973724365],
 [1962.0969038009644, 761.9254539012909, 30.53388786315918]]

In [16]:
plt.clf()

ax = plt.subplot(111)

fs = 17

# http://matplotlib.org/users/text_intro.html
ax.set_xlabel('l', fontsize=fs)
ax.set_ylabel('Objective Value', fontsize=fs)

plt.ylim(0,0.5)
plt.xlim(min(Lvalues), max(Lvalues))

xticks = [min(Lvalues)]
for l in range(10, max(Lvalues) + 10, 10):
    xticks.append(l)

ax.set_xticks(xticks)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(fs) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(fs) 

colors = ['ro', 'b^', 'gv']
labelNames = ['Replacement Greedy', 'Greedy Sum', 'Greedy Merge']

for ind in range(len(colors)-1, -1,-1):
    vals = []
    for i in range(len(runtimeRatio)):
        vals.append(runtimeRatio[i][ind])
        
    plt.plot(Lvalues, vals, c = colors[ind][0], marker = colors[ind][1], linewidth=2, label = labelNames[ind])

# http://matplotlib.org/1.3.0/examples/pylab_examples/legend_demo.html
legend = ax.legend(loc='lower right')

# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize(fs)

# plt.show()
plt.savefig("../../writeup/images/movielens-sublinear-runtime-ratio-fixed-k")

plt.close()